#Pruebas modelos (Full Dataset)

##Activación para Colab

In [0]:
#LMG: Variable para controlar si estamos en Colab o no y aplicar a las celdas:
inColab = True

En la siguiente celda, montamos la carpeta personal del Drive en Colab en caso de estar en este entorno. Pedirá un código al que se accede desde el enlace que facilita.

In [0]:
#LMG: Para Google CoLab tener el repo de Drive:
if inColab:
  from google.colab import drive
  drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##Carga de librerías y preparación de datos

Antes de preparar todo, elegimos el modelo a utilizar:

In [0]:
useDense = False
useLSTM = True

Ahora pasamos a la carga de librerías y preparación de datos:

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.optimizers import SGD
from keras.regularizers import l2

Using TensorFlow backend.


In [0]:
#LMG: Para CoLab en sample_data:
if inColab:
  data_consumo = pd.read_csv("/content/drive/My Drive/TFM/data/data_total.csv")
else:
  data_consumo = pd.read_csv("data_total.csv")
  
data_consumo.head()

,Unnamed: 0,fecha,PVPC_DEF,PVPC_2_PED_NOC,PVPC_ELEC_NOC,date_timestamp,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,date_day,Brent_price,Holiday,Precio de Regulación Secundaria subir,Precio de Regulación Secundaria bajar,Precio mercado SPOT Diario_x,Demanda real,Generación prevista Solar,Saldo total interconexiones programa p48,Generación programada P48 Exportación Portugal,Generación programada P48 Exportación Francia,Generación programada P48 Importación Portugal,Generación programada P48 Importación Francia,Precio SPOT PT,Precio SPOT FR
0,0,2015-10-01 00:00:00+02:00,117.77,69.49,71.88,1.443650e+09,24984.666667,1003.666667,6012.833333,402.166667,33.0,368.500000,580.166667,24691.833333,24517.0,2015-10-01,47.48,0.0,61.18,NaN,56.65,25305.0,267.6,182.4,-606.6,NaN,NaN,1200.0,56.65,38.56
1,1,2015-10-01 01:00:00+02:00,117.09,68.21,65.57,1.443654e+09,23550.833333,899.000000,6013.166667,174.833333,33.0,141.333333,584.166667,23440.000000,23169.0,2015-10-01,47.48,0.0,59.59,NaN,53.50,23959.0,195.7,-269.5,-1112.5,NaN,NaN,1200.0,53.50,31.27
2,2,2015-10-01 02:00:00+02:00,114.59,65.34,62.53,1.443658e+09,22648.166667,927.500000,6013.500000,103.833333,33.0,70.666667,585.833333,22521.166667,22437.0,2015-10-01,47.48,0.0,56.27,49.13,49.69,22873.0,202.1,94.0,-951.0,NaN,NaN,1200.0,49.69,30.66
3,3,2015-10-01 03:00:00+02:00,116.60,66.53,63.40,1.443661e+09,22203.833333,935.666667,6013.000000,102.000000,33.0,69.000000,588.500000,22335.333333,22281.0,2015-10-01,47.48,0.0,45.99,45.50,49.10,22550.0,61.3,319.7,-846.3,NaN,NaN,1200.0,49.10,29.14
4,4,2015-10-01 04:00:00+02:00,122.07,71.03,67.37,1.443665e+09,21987.833333,900.833333,6014.166667,101.666667,33.0,68.166667,589.000000,22177.833333,21910.0,2015-10-01,47.48,0.0,52.28,49.13,51.25,22017.0,45.1,405.5,-760.5,NaN,NaN,1200.0,51.25,29.40


In [0]:
x_data = data_consumo.drop(['PVPC_DEF','PVPC_2_PED_NOC','PVPC_ELEC_NOC'], axis=1).copy()
y_data = data_consumo[['PVPC_DEF','PVPC_2_PED_NOC','PVPC_ELEC_NOC']]

# Delete unconsistent columns and get the columns:
x_data = x_data.drop(columns=['fecha','date_day'])
x_data_columns = x_data.columns
y_data_columns = y_data.columns

x_data.head()

,Unnamed: 0,date_timestamp,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,Brent_price,Holiday,Precio de Regulación Secundaria subir,Precio de Regulación Secundaria bajar,Precio mercado SPOT Diario_x,Demanda real,Generación prevista Solar,Saldo total interconexiones programa p48,Generación programada P48 Exportación Portugal,Generación programada P48 Exportación Francia,Generación programada P48 Importación Portugal,Generación programada P48 Importación Francia,Precio SPOT PT,Precio SPOT FR
0,0,1.443650e+09,24984.666667,1003.666667,6012.833333,402.166667,33.0,368.500000,580.166667,24691.833333,24517.0,47.48,0.0,61.18,NaN,56.65,25305.0,267.6,182.4,-606.6,NaN,NaN,1200.0,56.65,38.56
1,1,1.443654e+09,23550.833333,899.000000,6013.166667,174.833333,33.0,141.333333,584.166667,23440.000000,23169.0,47.48,0.0,59.59,NaN,53.50,23959.0,195.7,-269.5,-1112.5,NaN,NaN,1200.0,53.50,31.27
2,2,1.443658e+09,22648.166667,927.500000,6013.500000,103.833333,33.0,70.666667,585.833333,22521.166667,22437.0,47.48,0.0,56.27,49.13,49.69,22873.0,202.1,94.0,-951.0,NaN,NaN,1200.0,49.69,30.66
3,3,1.443661e+09,22203.833333,935.666667,6013.000000,102.000000,33.0,69.000000,588.500000,22335.333333,22281.0,47.48,0.0,45.99,45.50,49.10,22550.0,61.3,319.7,-846.3,NaN,NaN,1200.0,49.10,29.14
4,4,1.443665e+09,21987.833333,900.833333,6014.166667,101.666667,33.0,68.166667,589.000000,22177.833333,21910.0,47.48,0.0,52.28,49.13,51.25,22017.0,45.1,405.5,-760.5,NaN,NaN,1200.0,51.25,29.40


In [0]:
# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.33, shuffle=False)

#Reshape for the LSTM
x_train = x_train.to_numpy()
x_valid = x_valid.to_numpy()
y_train = y_train.to_numpy()
y_valid = y_valid.to_numpy()

#Convert NaN values to 0:
x_train = np.nan_to_num(x_train)
x_valid = np.nan_to_num(x_valid)
y_train = np.nan_to_num(y_train)
y_valid = np.nan_to_num(y_valid)

print('Xtrain_dim:', x_train.shape)
print('Ytrain_dim:', y_train.shape)

Xtrain_dim: (20471, 25)
Ytrain_dim: (20471, 3)


Veamos cómo queda:

In [0]:
x_train_DF = pd.DataFrame(x_train, columns = x_data_columns)
x_train_DF.head()

,Unnamed: 0,date_timestamp,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,Brent_price,Holiday,Precio de Regulación Secundaria subir,Precio de Regulación Secundaria bajar,Precio mercado SPOT Diario_x,Demanda real,Generación prevista Solar,Saldo total interconexiones programa p48,Generación programada P48 Exportación Portugal,Generación programada P48 Exportación Francia,Generación programada P48 Importación Portugal,Generación programada P48 Importación Francia,Precio SPOT PT,Precio SPOT FR
0,0.0,1.443650e+09,24984.666667,1003.666667,6012.833333,402.166667,33.0,368.500000,580.166667,24691.833333,24517.0,47.48,0.0,61.18,0.00,56.65,25305.0,267.6,182.4,-606.6,0.0,0.0,1200.0,56.65,38.56
1,1.0,1.443654e+09,23550.833333,899.000000,6013.166667,174.833333,33.0,141.333333,584.166667,23440.000000,23169.0,47.48,0.0,59.59,0.00,53.50,23959.0,195.7,-269.5,-1112.5,0.0,0.0,1200.0,53.50,31.27
2,2.0,1.443658e+09,22648.166667,927.500000,6013.500000,103.833333,33.0,70.666667,585.833333,22521.166667,22437.0,47.48,0.0,56.27,49.13,49.69,22873.0,202.1,94.0,-951.0,0.0,0.0,1200.0,49.69,30.66
3,3.0,1.443661e+09,22203.833333,935.666667,6013.000000,102.000000,33.0,69.000000,588.500000,22335.333333,22281.0,47.48,0.0,45.99,45.50,49.10,22550.0,61.3,319.7,-846.3,0.0,0.0,1200.0,49.10,29.14
4,4.0,1.443665e+09,21987.833333,900.833333,6014.166667,101.666667,33.0,68.166667,589.000000,22177.833333,21910.0,47.48,0.0,52.28,49.13,51.25,22017.0,45.1,405.5,-760.5,0.0,0.0,1200.0,51.25,29.40


Por último, generamos funciones y variables de utilidad a lo largo de todos los modelos:

In [0]:
def rmse (y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred -y_true), axis=-1))

##Ejemplo de NN

Red neuronal densa como punto de partida.

###Preparación y entrenamiento

Primero, estandarizamos los **datos de entrenamiento**:

In [0]:
if useDense:
  scaler = MinMaxScaler()
  x_train_data = scaler.fit_transform(x_train)
  y_train_data = scaler.fit_transform(y_train)
  x_valid_data = x_valid
  y_valid_data = y_valid

  print('Entrenamiento:',x_train_data.shape, y_train_data.shape)
  print('Test:',x_valid_data.shape, y_valid_data.shape)

Entrenamiento: (20471, 25) (20471, 3)
Test: (10084, 25) (10084, 3)


Para visualizar (ver apartado Visualización de dataframes al final):

In [0]:
if useDense:
  x_train_data_DF = pd.DataFrame(x_train_data, columns = x_data_columns)

In [0]:
if useDense:
  y_train_data_DF = pd.DataFrame(y_train_data, columns = y_data_columns)

Por último, definimos los tamaños de nuestra red y algunas variables adicionales

In [0]:
if useDense:
  batch_size = 16
  num_var = x_train_data.shape[1]
  hidden_size = 100
  output_size = y_train_data.shape[1]

###Modelo

In [0]:
if useDense:
  # define model
  model = Sequential()
  model.add(Dense(hidden_size, 
                  activation='relu',
                  kernel_initializer='zeros',
                  input_shape=(num_var,),
                  kernel_regularizer=l2(1e-2)))
  model.add(Dropout(0.2))
  model.add(Dense(hidden_size, 
                  activation='relu',
                  kernel_initializer='zeros',
                  kernel_regularizer=l2(1e-2)))
  model.add(Dropout(0.2))
  model.add(Dense(output_size,
                  activation='linear',
                  kernel_initializer='zeros',
                  kernel_regularizer=l2(1e-2)))

  # compile mode
  #opt = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True)
  opt='adam'
  model.compile(optimizer=opt, loss='mse', metrics=['mse',rmse,'mae','mape'])
  print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 100)               2600      
_________________________________________________________________
dropout_27 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_28 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 3)                 303       
Total params: 13,003
Trainable params: 13,003
Non-trainable params: 0
_________________________________________________________________
None


###Entrenamiento

In [0]:
if useDense:
  history_train = model.fit(x_train_data, y_train_data, 
                            epochs=100, 
                            batch_size=batch_size, 
                            shuffle=False,
                            verbose=1)

  # summarize history for loss, MAPE in a different scale than the others
  plt.plot(history_train.history['mean_squared_error'])
  plt.plot(history_train.history['mean_absolute_error'])
  plt.plot(history_train.history['rmse'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['MSE','MAE','RMSE'], loc='upper left')
  plt.show()

  plt.plot(history_train.history['mean_absolute_percentage_error'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['MAPE'], loc='upper left')
  plt.show()

Epoch 1/100
20471/20471 [==============================] - 32s 2ms/step - loss: 0.0296 - mean_squared_error: 0.0296 - rmse: 0.1607 - mean_absolute_error: 0.1459 - mean_absolute_percentage_error: 13952.4387
Epoch 2/100
20471/20471 [==============================] - 30s 1ms/step - loss: 0.0277 - mean_squared_error: 0.0277 - rmse: 0.1579 - mean_absolute_error: 0.1431 - mean_absolute_percentage_error: 13902.5691
Epoch 3/100
20471/20471 [==============================] - 31s 1ms/step - loss: 0.0277 - mean_squared_error: 0.0277 - rmse: 0.1579 - mean_absolute_error: 0.1431 - mean_absolute_percentage_error: 13902.5691
Epoch 4/100
20471/20471 [==============================] - 30s 1ms/step - loss: 0.0277 - mean_squared_error: 0.0277 - rmse: 0.1579 - mean_absolute_error: 0.1431 - mean_absolute_percentage_error: 13902.5691
Epoch 5/100
20471/20471 [==============================] - 31s 1ms/step - loss: 0.0277 - mean_squared_error: 0.0277 - rmse: 0.1579 - mean_absolute_error: 0.1431 - mean_absolute

KeyboardInterrupt: ignored

##Ejemplo de LSTM

Basado en el ejemplo completo que está explicado en https://adventuresinmachinelearning.com/keras-lstm-tutorial/

###Preparación y entrenamiento

Para que se ejecute en paquetes, replicamos el dataset de n en n lotes:

In [0]:
if useLSTM:
  # split a sequence into samples
  def split_sequence(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
      # find the end of this pattern
      end_ix = i + n_steps
      # check if we are beyond the sequence
      if end_ix > len(sequence)-1:
        break
      # gather input and output parts of the pattern
      seq_x = sequence[i:end_ix,:]
      X.append(seq_x)

    return np.array(X)

Después, estandarizamos los **datos de entrenamiento** y aplicamos la función a la x e y, agrupando las salidas por paquetes de num_steps:

In [0]:
if useLSTM:
  scaler = MinMaxScaler()
  x_train_est = scaler.fit_transform(x_train)
  y_train_est = scaler.fit_transform(y_train)
  
  split_steps = 10
  x_train_data = split_sequence(x_train_est, split_steps)
  y_train_data = y_train_est[split_steps:,:]
  x_valid_data = split_sequence(x_valid, split_steps)
  y_valid_data = y_valid[split_steps:,:]

  print('Entrenamiento:',x_train_data.shape, y_train_data.shape)
  print('Test:',x_valid_data.shape, y_valid_data.shape)

Entrenamiento: (20461, 10, 25) (20461, 3)
Test: (10074, 10, 25) (10074, 3)


Para visualizar uno de los datos que van a entrar (ver apartado Visualización de dataframes al final):

In [0]:
if useLSTM:
  x_train_data_DF = pd.DataFrame(x_train_data[0,:,:], columns = x_data_columns)

In [0]:
if useLSTM:
  y_train_data_DF = pd.DataFrame(y_train_data, columns = y_data_columns)

Por último, definimos los tamaños de nuestra red y algunas variables adicionales:

En nuestro caso, la entrada va a ser de *1* **x** *nº de datos tomados* **x** *nº de variables independientes*, siendo:

- batch_size, para cada instante tomamos los datos en paquetes.
- nº de datos tomados o num_steps, esto es, en el análisis secuencial cada registro energético completo con el que contamos, o en términos más coloquiales, cuántas filas del dataset se tienen en cuenta .
- nº de variables independientes o num_var, cada registro en los datos de entrada, o en términos más coloquiales, cada columna del dataset.

Además:

- hidden_size, número de unidades en cada célula del LSTM.

- A la salida, tenemos los tres valores a estimar.

In [0]:
if useLSTM:
  batch_size = 16
  num_steps = split_steps
  num_var = x_train_data.shape[2]
  hidden_size = 100
  output_size = y_train_data.shape[1]

###Modelo

A la capa LSTM se le pasa cada vez un instante, siendo el primero el t1, el segundo el t2, etc.

Respecto al ejemplo en la página, hemos:
- Quitado el embedding inicial, ya que no necesitamos codificar la entrada (ya son valores en sí mismos).
- Cambiado el hidden-layer-size. En la página dicen que se suele poner al tamaño de entrada de cada registro. Viene a ser el símil de unidades en una capa densa.
- Dejado una única capa LSTM.
- Eliminado el dropout.
- TimeDistributed usa una capa densa para cada step del entrenamiento. La quitamos también y dejamos la densa exclusivamente, ya que hace la salida muy grande.

Así:


In [0]:
if useLSTM:
  # define model
  model = Sequential()
  model.add(LSTM(hidden_size, 
                 activation='relu', 
                 return_sequences=True, 
                 input_shape=(num_steps,num_var),
                 kernel_regularizer=l2(1e-2)))
  model.add(Dropout(0.2))
  model.add(LSTM(hidden_size, 
                 activation='relu',
                 kernel_regularizer=l2(1e-2)))
  model.add(Dropout(0.2))
  model.add(Dense(output_size,
                 activation='relu',
                 kernel_regularizer=l2(1e-2)))

  # compile mode
  opt = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True)
  model.compile(optimizer=opt, loss='mse', metrics=['mse',rmse,'mae','mape'])
  print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 10, 100)           50400     
_________________________________________________________________
dropout_11 (Dropout)         (None, 10, 100)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dropout_12 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 303       
Total params: 131,103
Trainable params: 131,103
Non-trainable params: 0
_________________________________________________________________
None


###Entrenamiento

In [0]:
if useLSTM:
  history_train = model.fit(x_train_data, y_train_data, 
                            epochs=2, 
                            batch_size=batch_size, 
                            shuffle=False,
                            verbose=1)

  # summarize history for loss, MAPE in a different scale than the others
  plt.plot(history_train.history['mean_squared_error'])
  plt.plot(history_train.history['mean_absolute_error'])
  plt.plot(history_train.history['rmse'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['MSE','MAE','RMSE'], loc='upper left')
  plt.show()

  plt.plot(history_train.history['mean_absolute_percentage_error'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['MAPE'], loc='upper left')
  plt.show()

Epoch 1/2
11760/20461 [================>.............] - ETA: 11s - loss: 0.3827 - mean_squared_error: 0.0287 - rmse: 0.1593 - mean_absolute_error: 0.1457 - mean_absolute_percentage_error: 23398.9065

High MAPE: I think it's because we are calling MinMaxScaler with a range of 0-1: scaler = MinMaxScaler(feature_range=(0, 1)) dataset = scaler.fit_transform(dataset) This guarantees that the minimum value of y_true will be 0. Because it can't divide by 0 it has to treat 0 as K.epsilon, returning a large number instead of infinity. – reductionista May 30 at 22:13 
You are right, I am using that standardization, so another solution would be to use mae since in the [0-1] domain is equivalent to mape.

###Test

Para el test:

In [0]:
if useLSTM:
  scores_test = model.evaluate(x_valid_data, y_valid_data, batch_size=batch_size, verbose=1)

  # summarize loss
  print("%s: %.2f" % (model.metrics_names[1], scores_test[1]))

In [0]:
if useLSTM:
  x_valid_data_DF = pd.DataFrame(x_valid_data[0,:,:], columns = x_data_columns)
  x_valid_data_DF.head()

In [0]:
if useLSTM:
  print(y_valid_data[0:1,:].shape)

  y_valid_data_DF = pd.DataFrame(y_valid_data[0:1,:], columns = y_data_columns)
  y_valid_data_DF.head()

In [0]:
if useLSTM:
  model.predict(x_valid_data[0:9,:,:], verbose=1)

##Visualización de dataframes

Datos de entranamiento:

In [0]:
x_train_data_DF.head()

In [0]:
y_train_data_DF.head()